In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from state import get_state, queue
import os
import traci

In [3]:
from dotenv import load_dotenv
load_dotenv()


sumo_bin = os.getenv("SUMO")
sumo_gui_bin = os.getenv("SUMO-GUI")
simulConfig = os.getenv("SIMUL-CONFIG")

In [4]:
import numpy as np
from tensorflow.keras import Sequential, layers
from tensorflow.random import set_seed
from collections import deque
from tensorflow import keras
from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt
import tensorflow as tf

In [5]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [48]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

/Users/arseneclaustre/.pyenv/versions/3.10.6/envs/RL_traffic/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/Users/arseneclaustre/.pyenv/versions/3.10.6/envs/RL_traffic/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:
        Q_values = model_action.predict(state[np.newaxis], verbose=0)[0]
        print(f"state : {state}")
        print(f"Q_values : {Q_values}")
        return Q_values.argmax()  # optimal action according to the DQN

In [7]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [8]:
batch_size = 32
discount_factor = 0.5
optimizer = keras.optimizers.Adam(learning_rate=0.05)
loss_fn = MeanSquaredError()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model_action.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model_action(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model_action.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_action.trainable_variables))

In [9]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
replay_buffer = deque(maxlen=10000)


sumoCmd = [sumo_bin, "-c", simulConfig, "--start"]
sumoCmd = [sumo_bin, "-c", simulConfig, "--start", "--no-warnings"]

for episode in range(300):
    if traci.isLoaded():
        traci.close()
    traci.start(sumoCmd)
    lane_ids =  traci.lane.getIDList()
    # print(lane_ids[0])

    trafic_light_ids = traci.trafficlight.getIDList()

    # state = np.array(queue(lane_ids))
    state=np.array(get_state(lane_ids))
    action=-1
    # print(state)
    wait_times.append(0)
    for step in range(10000): ## TO CHANGED
        epsilon = max(1 - episode / 300, 0.01)

        if step%1000 == 0:
            #print(f"longeur du buffer :{len(replay_buffer)}")
            ########################################################################
            ##Calcul de la reward
            # next_state = np.array(queue(lane_ids))
            next_state = np.array(get_state(lane_ids))
            # reward = calculate_reward(values, reward, total_reward)[0]
            # reward=(np.sum(state)-np.sum(next_state))
            reward = np.sum(state[:24])- np.sum(next_state[:24])
            replay_buffer.append((state, action, reward, next_state))

            list_values.append(queue(lane_ids))
            #########################################################################
            state=next_state
            action = epsilon_greedy_policy(state, epsilon)
            #print("action", action)
            traci.trafficlight.setPhase(trafic_light_ids[0],2*action)

            if len(replay_buffer) >= batch_size*10:
                training_step(batch_size)
                # new_weights = model_action.get_weights()
                # weights.append(new_weights)
                #print(f"Episode {episode}: new weights = {new_weights}")
            # else:
            #     print(f"Episode {episode}: pas assez de données dans le replay buffer.")
            # print(values)
            # if list_values:
            #    # print(f'list values {list_values[-1]}')


        traci.simulationStep()
    print(f'episode : {episode}')
    traci.close()


 Retrying in 1 seconds


episode : 0
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 84 ACT 41 BUF 0)                    
 Retrying in 1 seconds


episode : 1
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 84 ACT 46 BUF 0)                    
 Retrying in 1 seconds


state : [ 0  0  6  0  0  0  0  0  2  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0 10  2  2  2  4  0  2  0  6  0  0  1  1  3  0  1  0  1  1  0  0  0]
Q_values : [ 0.19114468  0.8559715  -0.6384022  -0.16767056]
episode : 2
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 84 ACT 39 BUF 0)                    
 Retrying in 1 seconds


episode : 3
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 84 ACT 46 BUF 0)                    
 Retrying in 1 seconds


...

state : [ 0  0  0  0  2  2  0  0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  1  4  2  4  4  4  0  0  1 11  0  0  0  0  3  0 10  0  0  0  0  0  1]
Q_values : [ 0.56639296  0.5793751  -0.7468986   0.5749526 ]
episode : 25
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 84 ACT 48 BUF 0)                    
 Retrying in 1 seconds


episode : 26
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 84 ACT 45 BUF 0)                    
 Retrying in 1 seconds


state : [0 2 0 0 2 1 0 2 0 0 3 1 0 0 0 0 0 0 0 0 0 0 0 0 1 2 3 2 2 1 6 2 0 1 7 1 2
 0 1 2 0 3 2 0 1 2 0 2]
Q_values : [-0.10465995  0.6197741  -0.3969926  -0.08399945]
episode : 27
Step #100.00 (1ms ~= 10.00*RT, ~46000.00UPS, TraCI: 0ms, vehicles TOT 84 ACT 46 BUF 0)     
 Retrying in 1 seconds


state : [ 0  0  3  0  3  0  0  0  0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  7  0  4  0  4  0  0  0 12  0  2  0  2  4  0  5  0  0  3  0  0  1]
Q_values : [ 0.8300604   0.95768875 -0.21402703  0.28669086]
Interrupt signal received, trying to exit gracefully.s, vehicles TOT 64 ACT 48 BUF 8)     


KeyboardInterrupt: 

In [12]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
# replay_buffer = deque(maxlen=2000)


sumoCmd = [sumo_gui_bin, "-c", simulConfig, "--start"]

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
# print(lane_ids[0])
# for lane in lane_ids:
#     print(traci.lane.getLastStepVehicleNumber(lane))
# north_lane = traci.lane.getLastStepVehicleNumber("N_0")
# south_lane = traci.lane.getLastStepVehicleNumber("S_0")
# east_lane = traci.lane.getLastStepVehicleNumber("E_0")
# west_lane = traci.lane.getLastStepVehicleNumber("W_0")
trafic_light_ids = traci.trafficlight.getIDList()

state = np.array(get_state(lane_ids))
action=-1
# print(state)
wait_times.append(0)
for step in range(10000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)
    nom_du_feu= traci.trafficlight.getIDList()[0]

    if step%500 == 0:
        state=np.array(get_state(lane_ids))
        action = epsilon_greedy_policy(state,0)*2
        # action = np.random.randint(8)
        print("action", action)
        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu))
        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu)[0].phases[action])
        traci.trafficlight.setPhase(trafic_light_ids[0],action)
    traci.simulationStep()

traci.close()


 Retrying in 1 seconds
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [0. 0. 0. 0.]
action 0
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 1 0 3 1 0 3 1 0 3 1 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.40326726  0.91583157  0.9705516   0.06993382]
action 4
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 2 1 3 2 1 3 2 1 3 2 1 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.6157905   0.77695376  0.8243077   0.32392374]
action 4
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 2 2 3 1 4 3 1 3 4 1 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.85071224  0.6993439   0.6738202   0.74666685]
action 6
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 3 2 2 2 4 2 2 3 3 2 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.50390565  0.33347192  0.25533578  0.990328  ]
action 6
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 2 4 2 2 2 5 2 2 3 4 2 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.887483